In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_groq import ChatGroq

model = ChatGroq(model="openai/gpt-oss-120b", temperature=0)

## Creating subagents

In [2]:
from langchain.tools import tool

@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x ** 0.5

@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x ** 2

In [4]:
from langchain.agents import create_agent

# create subagents

subagent_1 = create_agent(
    model=model,
    tools=[square_root]
)

subagent_2 = create_agent(
    model=model,
    tools=[square]
)

## Calling subagents

In [5]:
from langchain.messages import HumanMessage

@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke({"messages": [HumanMessage(content=f"Calculate the square root of {x}")]})
    return response["messages"][-1].content

@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke({"messages": [HumanMessage(content=f"Calculate the square of {x}")]})
    return response["messages"][-1].content

## Creating the main agent

main_agent = create_agent(
    model=model,
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.")

## Test

In [6]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [7]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='731b1b25-ff01-42a4-9d49-84100eaf40f9'),
              AIMessage(content='', additional_kwargs={'reasoning_content': 'We need to compute sqrt(456). Use subagent 1.', 'tool_calls': [{'id': 'fc_0e052b8e-2f4e-48e8-9d8e-73eaf5b3c71c', 'function': {'arguments': '{"x":456}', 'name': 'call_subagent_1'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 198, 'total_tokens': 243, 'completion_time': 0.093186252, 'completion_tokens_details': {'reasoning_tokens': 15}, 'prompt_time': 0.00787606, 'prompt_tokens_details': None, 'queue_time': 0.004496324, 'total_time': 0.101062312}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_7ee00986fe', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c49ad-ee63-7293-a171-7141162ce1e0-0', tool_calls=[{'name': 'cal